# Рак Алексей. Курс 4. Группа 3.
# Лабораторная работа № 3

# Условие

Вычислить интеграл:

\begin{gather}
I = \int\limits_0^9 \frac{\cos^2 x}{1 + x^2} dx
\end{gather}

используя простейший метод Монте-Карло и метод симметризации подынтегральной функции. Сравнить дисперсии этих оценок (с помощью аналитических выкладок).

# Ход работы

Будем моделировать базовую случайную величину мультипликативным конгруэнтным методом:

$\alpha_0^* = \beta = 65539$

$M = 2^{31}$

$\alpha_i^* = \{\beta \alpha_{i - 1}^*\} \mod M$

$\alpha_i = \alpha_i^* / M$

Далее моделируем случайную величину из равномерного распределения методом обратной функции:

$\mathbb{U}(a, b) = \frac{\alpha_i}{b - a} + a$

Далее используем простейший метод Монте-Карло, для этого моделируем случайную величину из равномерного распределения на отрезке от  $0$ до $9$. И находим значение функции $\frac{\cos^2 x}{1 + x^2}$ в смоделированной точке.

Также используем метод симметризации подынтегральной функции. В этом случае опять используем  метод Монте-Карло, но в этом случае уже находим значение функции $\frac{1}{2}\left(\frac{\cos^2 x}{1 + x^2} + \frac{\cos^2 (9 - x)}{1 + (9 - x)^2}\right)$

# Программная реализация

In [99]:
import numpy as np

In [105]:
low = 0
hight = 9

In [1]:
class BRV:
    def __init__(self):
        self.seed = 1
        self.beta = 65539
        self.M = 2 ** 31
        
    def __call__(self):
        self.seed = (self.seed * self.beta) % self.M
        return self.seed / self.M

In [63]:
class URV(BRV):
    def __init__(self, a, b):
        super().__init__()
        self.a = a
        self.b = b
        
    def __call__(self):
        return super().__call__() * (self.b - self.a) + self.a

In [120]:
def montecarlo(func, samples=10000):
    rv = URV(low, hight)
    answer = 0
    for _ in range(samples):
        answer += func(rv())
    return answer / samples * (hight - low)

In [121]:
def func(x):
    return (np.cos(x) ** 2) / (1 + x ** 2)

In [122]:
def symmetry_func(x):
    return (func(x) + func(low + hight - x)) / 2

# Результаты

In [123]:
print("simple = {}".format(montecarlo(func)))
print("symmetry = {}".format(montecarlo(symmetry_func)))

simple = 0.8269383491590565
symmetry = 0.8289740685498264


# Вывод

Сравним дисперсии двух полученных оценок:

$\mathbb{D} \{\xi\} = \mathbb{M}\{\xi^2\} - \left(\mathbb{M}\{\xi\}\right)^2$ 

$\xi_1 = \frac{\cos^2 x}{1 + x^2}$

$\xi_2 = \frac{1}{2}\left(\frac{\cos^2 x}{1 + x^2} + \frac{\cos^2 (9 - x)}{1 + (9 - x)^2}\right)$

$x = \mathbb{U}(0, 9)$

$\mathbb{M}\{\xi_1\} = \int\limits_0^9\frac{\cos^2 x}{1 + x^2}dx$

$\mathbb{M}\{\xi_2\} = \int\limits_0^9\frac{1}{2}\left(\frac{\cos^2 x}{1 + x^2} + \frac{\cos^2 (9 - x)}{1 + (9 - x)^2}\right)dx = \frac{1}{2}\int\limits_0^9\frac{\cos^2 x}{1 + x^2}dx + \frac{1}{2}\int\limits_0^9\frac{\cos^2 (9 - x)}{1 + (9 - x)^2}dx = \frac{1}{2}\int\limits_0^9\frac{\cos^2 x}{1 + x^2}dx - \frac{1}{2}\int\limits_9^0\frac{\cos^2 x}{1 + x^2}dx = \int\limits_0^9\frac{\cos^2 x}{1 + x^2}dx$

$\mathbb{M}\{\xi_1\} = \mathbb{M}\{\xi_2\}$

$\mathbb{M}\{\xi_1^2\} = \int\limits_0^9\frac{\cos^4 x}{(1 + x^2)^2}dx$

$\mathbb{M}\{\xi_2^2\} = \frac{1}{4}\int\limits_0^9\left(\frac{\cos^2 x}{1 + x^2} + \frac{\cos^2 (9 - x)}{1 + (9 - x)^2}\right)^2dx = \frac{1}{4}\int\limits_0^9\frac{\cos^4 x}{(1 + x^2)^2}dx + \frac{1}{2}\int\limits_0^9\frac{\cos^2 x}{1 + x^2}\frac{\cos^2 (9 - x)}{1 + (9 - x)^2}dx + \frac{1}{4}\int\limits_0^9\frac{\cos^4 (9 - x)}{(1 + (9 - x)^2)^2}dx = \frac{1}{2}\mathbb{M}\{\xi_1^2\} + \frac{1}{2}\int\limits_0^9\frac{\cos^2 x}{1 + x^2}\frac{\cos^2 (9 - x)}{1 + (9 - x)^2}dx \leq$

$ \leq \frac{1}{2}\mathbb{M}\{\xi_1^2\} + \frac{1}{4}\int\limits_0^9\frac{\cos^4 x}{(1 + x^2)^2}dx + \frac{1}{4}\int\limits_0^9\frac{\cos^4 (9 - x)}{(1 + (9 - x)^2)^2}dx = \mathbb{M}\{\xi_1^2\}$

$\mathbb{M}\{\xi_1^2\} \geq \mathbb{M}\{\xi_2^2\}$

$\mathbb{D}\{\xi_1\} \geq \mathbb{D}\{\xi_2\}$

Однако стоит отметить, что несмотря на то, что дисперсия при использовании метода симметризации подынтегральной функции меньше, что при использовании простейшего метода Монте-Карло, трудоёмкость вычислений возрастает из-за того, что на каждой итерации нужно вычислить значение функции в двуъ точках, а не в одной.